Lo que tiene que hacer:
- Descartar las películas que no tienen “overview”. 
- Descartar las películas cuyo idioma original tenga más de 2 caracteres.
- Tomar las 100 palabras más comunes de todos los overviews combinados.
- Para generar el ”Overview” de cada película para el archivo de salida se deberán descartar las palabras que se encuentren dentro de las 100 más comunes de todos los overviews y se deberá generar un string con 3 palabras tomadas al azar y ordenadas al azar (ver random.sample()) de ese resultado.
- Se utilizarán como datos de las tarjetas: “Genre”, “Original_Language”, “Vote_Average”, “Release_Date” y 3 palabras aleatorias del overview separadas por “;”. Como dato a adivinar se utilizará “Title”. Descartar el resto de las columnas.
- El archivo resultante deberá tener las siguientes columnas (en este orden específico): “Genre”, “Original_Language”, “Release_Date”,“Vote_Average”, “Overview” y “Title”.

Una de las filas del dataset se encuentra dividia en varias lineas, entonces esto las junta en una sola linea y elimina las restantes.

In [1]:
import os
import csv
import collections
import random
import string

def ordenar_datos(lista):
    """Ordena una seguidilla de lineas que forman parte de una sola: desde el 1105 al 1115"""
    for i in range(1106,1115): 
        lista[1105][2] += lista[i][0]
    lista[1105][2] = lista[1105][2] + lista[1115][0]
    for i in range(1, len(lista[1115])): 
        lista[1105].append(lista[1115][i])
    for i in range(1115,1105,-1):
        lista.pop(i)
    return lista

Limpia los overviews de ciertos caracteres que no son letras.

In [2]:
def limpiar(cadena):
    """Elimina una serie de caracteres de la cadena ingresada."""
    for c in string.punctuation:
        cadena = cadena.replace(c, "")
    return cadena

Filtra los datos según si tienen overview y si el idioma es menor o igual a dos caracteres. Además se guardan las cien palabras más utilizadas en la suma de los overviews.

In [3]:
def filtrar(datos):
    #Filtro las peliculas por el overview y guardo las palabras que aparecen
    lista = []
    overview = ""
    for a in datos:
        if datos[2] != "":
            if len(datos[6]) <= 2: 
                lista.append([datos[7], datos[6], datos[0], datos[5], datos[2], datos[1]])
            overview += f"{datos[2]} "
    
    #Ordeno las palabras segun las veces que aparecen y tomo las 100 más repetidas.
    overview = sorted(dict(collections.Counter(limpiar(overview).split())).items(), key=lambda x: x[1])[-100:]
    overview = [x[0] for x in overview]
    return lista, overview

Devuelve tres palabras random del overview de la pelicula.

In [4]:
def filtrar_cien(cad, cien):
    """Elimina las palabras que estén en los cien y toma 3"""
    cad = limpiar(cad).split()
    cad = [c for c in cad if not c in cien]
    if len(cad) >= 3:
        cad = str(random.sample(cad,3))
    else:
        cad = str(random.sample(cad,len(cad)))
    return cad.replace("[","").replace("]","").replace(",",";").replace("'","").replace('"',"")

Codigo Principal.  
Se utilizan las funciones anteriores con el dataset para generar otro que esté filtrado con los puntos que se pidieron.

In [6]:
try:
    path_dataset = os.path.join(os.getcwd(), "..", "csv_originales", "mymoviedb.csv")
    with open(path_dataset, "r", encoding="utf-8") as dataset:
        csvreader = csv.reader(dataset, delimiter=",")
        head_csv, data_csv = next(csvreader), list(csvreader)

        ordenar_datos(data_csv)
        filtrado, cien_palabras = filtrar(data_csv)
        
    path_dataset = os.path.join(os.getcwd(), "..", "dataset_peliculas.csv")
    with open(path_dataset, "w", encoding="utf-8", newline="") as new_dataset:
        csvwriter = csv.writer(new_dataset, delimiter=",")
            
        csvwriter.writerow(["Genre", "Original_Language", "Release_Date", "Vote_Average", "Overview", "Title"])
        for dato in filtrado:
            dato[4] = filtrar_cien(dato[4], cien_palabras)
            csvwriter.writerow(dato)
            
except FileNotFoundError:
    print("El archivo mymoviedb.csv no se encontró")